In [1]:
.libPaths(c('~/miniconda3/envs/metadoc/lib/R//library','~/miniconda3/envs/Rdoc/lib/R//library'))

In [2]:
setwd('/import/DB/minidir/a.moth10/GC/')
# library(xlsx)
# con = read.xlsx("./ss.xlsx",sheetIndex = 1,header =T)
# con = con[,1:84]
# EGC = read.xlsx("./ss.xlsx",sheetIndex = 2,header =T)
# EGC = EGC[,1:84]
# GC = read.xlsx("./ss.xlsx",sheetIndex = 3,header =T)
# GC = GC[,1:84]
# library(tableone) 
# library(dplyr)
# data = rbind(con,EGC,GC) 
# data = data[!duplicated(data$patient.ID),2:84]

In [4]:
library(dplyr)

In [19]:
data = readRDS('./data.rds') %>%  filter(Group != 'Control')

In [20]:
data  = data[,c(1,76,78:84)]

In [21]:
dim(data)

[1] 1036    9

In [26]:
library(dplyr)

data1 = data %>%
  mutate(Size = case_when(Size < 2 ~ '<2 ',
                             Size > 2 & Size < 5 ~ '2-5',
                          Size  > 5 ~ '>5 ',
                             TRUE ~ NA ))


In [29]:
head(data1,2)

,Group,Size,area,degree.of.differentiation,physiology,Lymph.Node.Metastasis,Depth.of.Invasion,N_Stage,M_Stage
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,EGC,<2,A,G1G2,AC,No,T1,N0,Mx
2,EGC,2-5,B,G2,AC,No,T1,N0,M0


In [33]:
library(dplyr)
# data1 = data1 %>% tibble::rownames_to_column(.,'id')
# data1$id = paste0('T',data1$id)


delete.na <- function(DF, n=0) {
  DF[rowSums(is.na(DF)) <= n,]
}
data1 = delete.na(data1, 80)


library(pROC)
require(xgboost)
require(Matrix)
require(data.table)
if (!require('vcd')) {
  install.packages('vcd')
}
library(caret)   #
library(openxlsx) #

In [34]:
df_all = data1

In [35]:
# 
ohe_feats = c( "Size", "area" ,"degree.of.differentiation",
               "physiology","Lymph.Node.Metastasis",
               "Depth.of.Invasion","N_Stage",'M_Stage')

dummies <- dummyVars(~ Size +  area + degree.of.differentiation +
                       physiology +Lymph.Node.Metastasis +Depth.of.Invasion +N_Stage +  M_Stage
                     , data = df_all,fullRank=T)

In [36]:

df_all_ohe <- as.data.frame(predict(dummies, newdata = df_all))

df_all_combined <- cbind(df_all[,-c(which(colnames(df_all) %in% ohe_feats))], df_all_ohe)

In [41]:
colnames(df_all_combined)[1] = 'Group'

In [43]:
library(tidyverse)
set.seed(1123)
library(missForest)
dat2 <- missForest(df_all_combined,ntree = 100)
head(dat2$ximp)
dat2$OOBerror

Warning message:
"package 'tibble' was built under R version 4.2.2"
Warning message:
"package 'readr' was built under R version 4.2.3"
Warning message:
"package 'purrr' was built under R version 4.2.2"
Warning message:
"package 'stringr' was built under R version 4.2.2"
Warning message:
"package 'forcats' was built under R version 4.2.3"
Warning message:
"package 'lubridate' was built under R version 4.2.3"
-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v forcats   1.0.0     v stringr   1.5.0
v lubridate 1.9.2     v tibble    3.2.1
v purrr     1.0.2     v tidyr     1.2.1
v readr     2.1.4     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x data.table::between() masks dplyr::between()
x tidyr::expand()       masks Matrix::expand()
x dplyr::filter()       masks stats::filter()
x data.table::first()   masks dplyr::first()
x lubridate::hour()     masks data.table::hour()
x lubridate::isoweek()  masks data.table::isoweek(

,Group,Size<2,Size>5,area.AB,area.A_B,area.B,area.BA,area.G,area.GA,area.GAA,...,physiology.mMA,Lymph.Node.Metastasis.Yes,Depth.of.Invasion.T2,Depth.of.Invasion.T3,Depth.of.Invasion.T4,N_Stage.N1,N_Stage.N2,N_Stage.N3,M_Stage.M1,M_Stage.Mx
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,EGC,1,0,0,0,0,0,0,0,0,...,0,0,0.00000000,0.0000000000,0.000000e+00,0,0,0,0,1
2,EGC,0,0,0,0,1,0,0,0,0,...,0,0,0.00000000,0.0000000000,0.000000e+00,0,0,0,0,0
3,EGC,1,0,0,0,1,0,0,0,0,...,0,0,0.00000000,0.0000000000,0.000000e+00,0,0,0,0,0
4,EGC,1,0,0,0,0,0,0,0,0,...,0,0,0.00000000,0.0000000000,0.000000e+00,0,0,0,0,0
5,EGC,1,0,0,0,0,0,0,1,0,...,0,0,0.08325476,0.0007537784,6.418477e-18,0,0,0,0,0
6,EGC,1,0,0,0,1,0,0,0,0,...,0,0,1.00000000,0.0000000000,0.000000e+00,0,0,0,0,0


NRMSE       PFC 
0.5180463 0.0000000

In [44]:
df_all_combined2 = dat2$ximp

In [47]:
df_all_combined2$Group = ifelse(df_all_combined2$Group  == 'EGC',1,0)

In [49]:
saveRDS(df_all_combined2,file = 'df_all_combined.egc.gc.rds')

In [50]:
saveRDS(data1,file = 'df_all_combined.egc.gc2.rds')